<a href="https://colab.research.google.com/github/goodakai/GEE/blob/main/GEE_mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# python code for mapping leucogranite based on GEE
# author: Dakai Guo,Ziye Wang
# contact: Ziye Wang (Email: ziyewang@cug.edu.cn)

# mount google cloud drive and GEE API
from google.colab import drive
drive.mount('/content/drive')
import ee
ee.Authenticate()
ee.Initialize(project='ee-kaifangguo')

# input vector file
aoi = ee.FeatureCollection("projects/ee-kaifangguo/assets/AOI")
hl_aoi = ee.FeatureCollection("projects/ee-kaifangguo/assets/cndhl")
unhl_aoi = ee.FeatureCollection("projects/ee-kaifangguo/assets/unhl")

# Sentinel2 cloud mask function
def maskS2clouds(image):
  qa = image.select('QA60');
  cloudBitMask = 1 << 10;
  cirrusBitMask = 1 << 11;
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0));
  return image.updateMask(mask).divide(100);

# input ASTER and Sentinel2 image
asterimage =  (ee.Image('projects/ee-kaifangguo/assets/asterjz')
             .multiply(100)
             .clip(aoi)
             .select(['green','red','nir','swir1','swir2','swir3','swir4','swir5','swir6'],
                 ['A01','A02','A3N','A04','A05','A06','A07','A08','A09']));

sentinel2image =  (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                   .filterDate('2020-01-01','2022-12-31')
                   .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10))
                   .filterBounds(aoi)
                   .map(maskS2clouds)
                   .median()
                   .clip(aoi)
                   .select(['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']));

# select remote sensing bands
asterbands = asterimage.select('A3N','A04','A06','A07')
sentinel2bands = sentinel2image.select('B11')
alldata = asterbands.addBands(sentinel2bands)
bands = alldata.bandNames()

# production of train data
positive_po = ee.FeatureCollection.randomPoints(hl_aoi,4000)
negative_po = ee.FeatureCollection.randomPoints(unhl_aoi,4000)

def add_1(feature):
    return feature.set('Lithology', 1)
positive_data = positive_po.map(add_1)

def add_0(feature):
    return feature.set('Lithology', 0)
negative_data = negative_po.map(add_0)

train_points = positive_data.merge(negative_data)
train_data = alldata.sampleRegions(**{
    'collection': train_points,
    'properties': ['Lithology'],
    'scale': 100
})

# training the random forest model
withRandom = train_data.randomColumn('random')
split = 0.7
trainingPartition = withRandom.filter(ee.Filter.lt('random', split))
testingPartition = withRandom.filter(ee.Filter.gte('random', split))

rf = ee.Classifier.smileRandomForest(15).setOutputMode('PROBABILITY').train(**{
    'features': trainingPartition,
    'classProperty': 'Lithology',
    'inputProperties': bands
})

# test model performance
validated = testingPartition.classify(rf)

def probToClass(feature):
    prob = ee.Number(feature.get('classification'))
    label = prob.gte(0.5).toInt()
    return feature.set('predicted', label)

validatedClass = validated.select('Lithology','classification').map(probToClass)
confusionMatrix = validatedClass.errorMatrix('Lithology','predicted')
print('confusionMatrix',confusionMatrix.getInfo())
print('overall accuracy', confusionMatrix.accuracy().getInfo())
print('kappa accuracy', confusionMatrix.kappa().getInfo())

# output the mapping results
img_classfication = alldata.classify(rf)
task = ee.batch.Export.image.toDrive(image=img_classfication,
                   region=aoi.geometry(),
                   description='5bands',
                   folder='hlrf',
                   fileNamePrefix='5bands',
                   scale=100,
                   crs='EPSG:4326')

task.start()